<a href="https://colab.research.google.com/github/JeraldYik/rcp-notebooks/blob/main/Final_FL_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip list -v | grep tensorflow

tensorflow                    1.15.2         /usr/local/lib/python3.7/dist-packages pip      
tensorflow-datasets           4.0.1          /usr/local/lib/python3.7/dist-packages pip      
tensorflow-estimator          2.4.0          /usr/local/lib/python3.7/dist-packages pip      
tensorflow-gcs-config         2.4.0          /usr/local/lib/python3.7/dist-packages pip      
tensorflow-hub                0.11.0         /usr/local/lib/python3.7/dist-packages pip      
tensorflow-metadata           0.29.0         /usr/local/lib/python3.7/dist-packages pip      
tensorflow-privacy            0.5.2          /usr/local/lib/python3.7/dist-packages pip      
tensorflow-probability        0.12.1         /usr/local/lib/python3.7/dist-packages pip      


In [2]:
!pip install tensorflow==1.15.2
!pip install tensorflow_privacy

  Using cached https://files.pythonhosted.org/packages/de/62/2ee9cd74c9fa2fa450877847ba560b260f5d0fb70ee0595203082dafcc9d/tensorflow_estimator-1.15.1-py2.py3-none-any.whl
ERROR: tensorflow-privacy 0.5.2 has requirement tensorflow-estimator>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Using cached https://files.pythonhosted.org/packages/74/7e/622d9849abf3afb81e482ffc170758742e392ee129ce1540611199a59237/tensorflow_estimator-2.4.0-py2.py3-none-any.whl
ERROR: tensorflow 1.15.2 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.4.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


In [3]:
import tensorflow as tf
import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

TensorFlow is already loaded. Please restart the runtime to change versions.


In [4]:
class Cloud:

  def __init__(self, num_clients):
    self.EPOCHS = 10
    self.BATCH_SIZE = 125
    self.LEARNING_RATE = 0.1
    self.L2_NORM_CLIP = 1.0
    self.num_clients = num_clients
    self.model = None
    self.loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.losses.Reduction.NONE)
  
  def generate_data(self):
    train, test = tf.keras.datasets.mnist.load_data()
    train_data, train_labels = train
    test_data, test_labels = test

    train_len = len(train_data)//2
    test_len = len(test_data)//2

    ''' Half the data '''
    train_data = train_data[:train_len]
    test_data = test_data[:test_len]
    train_labels = train_labels[:train_len]
    test_labels = test_labels[:test_len]

    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

    train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
    test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

    train_labels = np.array(train_labels, dtype=np.int32)
    test_labels = np.array(test_labels, dtype=np.int32)

    train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
    test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

    assert train_data.min() == 0.
    assert train_data.max() == 1.
    assert test_data.min() == 0.
    assert test_data.max() == 1.

    ''' splitting data and labels into array with length=num_clients '''
    train_data_arr = [None for _ in range(self.num_clients)]
    test_data_arr = [None for _ in range(self.num_clients)]
    train_labels_arr = [None for _ in range(self.num_clients)]
    test_labels_arr = [None for _ in range(self.num_clients)]

    for i in range(self.num_clients):
      train_data_arr[i] = train_data[(len(train_data)//self.num_clients)*i:(len(train_data)//self.num_clients)*(i+1)]
      test_data_arr[i] = test_data[(len(test_data)//self.num_clients)*i:(len(test_data)//self.num_clients)*(i+1)]
      train_labels_arr[i] = train_labels[(len(train_labels)//self.num_clients)*i:(len(train_labels)//self.num_clients)*(i+1)]
      test_labels_arr[i] = test_labels[(len(test_labels)//self.num_clients)*i:(len(test_labels)//self.num_clients)*(i+1)]

    print(len(train_data), len(test_data), len(train_labels), len(test_labels))
    print(len(train_data_arr), len(test_data_arr), len(train_labels_arr), len(test_labels_arr))
    print(len(train_data_arr[0]), len(test_data_arr[0]), len(train_labels_arr[0]), len(test_labels_arr[0]))
    
    return train_data_arr, test_data_arr, train_labels_arr, test_labels_arr


  def generate_model(self):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, 8,
                              strides=2,
                              padding='same',
                              activation='relu',
                              input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(2, 1),
        tf.keras.layers.Conv2D(32, 4,
                              strides=2,
                              padding='valid',
                              activation='relu'),
        tf.keras.layers.MaxPool2D(2, 1),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    # model.compile(optimizer='sgd', loss=loss, metrics=['accuracy'])

    self.model = model

  def send_model(self):
    cloned_model = tf.keras.models.clone_model(self.model)
    cloned_model.compile(optimizer='sgd', loss=self.loss, metrics=['accuracy'])
    return cloned_model

  ''' receive new weights from aggregator and set to model '''
  def receive_and_save_weights(self, weights):
    # print('-- Saving weights... --')
    self.model.set_weights(weights)

In [5]:
class Client:

  def __init__(self, idx):
    self.idx = idx
    self.model = None
    self.train_data = None
    self.test_data = None
    self.train_labels = None
    self.test_labels = None
  
  def download_model(self,model):
    self.model = model

  def download_data_and_labels(self,train_data_arr, test_data_arr, train_labels_arr, test_labels_arr):
    self.train_data = train_data_arr[self.idx]
    self.test_data = test_data_arr[self.idx]
    self.train_labels = train_labels_arr[self.idx]
    self.test_labels = test_labels_arr[self.idx]
    # print(len(self.train_data), len(self.test_data), len(self.train_labels), len(self.test_labels))

  ''' fit the model with training data '''
  def train(self, epochs, batch_size):
    self.model.fit(self.train_data, self.train_labels, validation_data=(self.test_data, self.test_labels), epochs=epochs, batch_size = batch_size, verbose=0)
    # self.model.fit(self.train_data, self.train_labels, validation_data=(self.test_data, self.test_labels), epochs=epochs, batch_size = batch_size)

  def get_weights_from_model(self):
    return self.model.get_weights()

In [6]:
class Aggregator:
  def __init__(self, num_clients):
    self.weights_from_clients = [None for _ in range(num_clients)]
    self.num_clients = num_clients
  
  def get_weights_from_clients(self, client_idx, weights):
    # print('-- Received weights from Client {} --'.format(client_idx))
    self.weights_from_clients[client_idx] = weights

  def aggregate_weights(self):
    # print('--- Aggregating weights... ---')

    ''' Populate return list with empty numpy arrays of appropriate size '''
    new_weights = []
    for layer in self.weights_from_clients[0]:
      new_weights.append(np.zeros(shape=layer.shape))
    
    ''' Add value of weights from all clients at each layer '''
    for client_weights in self.weights_from_clients:
      for i, w in enumerate(client_weights):
        new_weights[i] += w

    ''' Find average value of weights at each layer '''
    for layer in new_weights:
      layer /= self.num_clients

    return new_weights

  def send_weights_to_cloud(self, cloud):
    aggregated_weights = self.aggregate_weights()
    cloud.receive_and_save_weights(aggregated_weights)

In [7]:
print('--- Preparing Cloud ---') 
NUM_CLIENTS = 2 
cloud = Cloud(num_clients=NUM_CLIENTS) 
train_data_arr, test_data_arr, train_labels_arr, test_labels_arr = cloud.generate_data() 

--- Preparing Cloud ---
30000 5000 30000 5000
2 2 2 2
15000 2500 15000 2500


In [8]:
for i in range(20): 
  print('--- Round {} ---'.format(i+1))
  cloud.generate_model() 
  clients = [None for _ in range(cloud.num_clients)]

  # print('--- Preparing Aggregator ---') 
  aggregator = Aggregator(cloud.num_clients)

  ''' Populate clients array and save static data into clients ''' 
  for j in range(len(clients)): 
    clients[j] = Client(j) 
    clients[j].download_data_and_labels(train_data_arr, test_data_arr, train_labels_arr, test_labels_arr) 

  NUM_GENERATION = 3

  val_acc_epoch_start = []
  val_acc_epoch_end = []

  for j in range(NUM_GENERATION): 
    for client in clients:
      print('\n---- Generation {}. For Client {} ----'.format(j+1,client.idx))
      client.download_model(cloud.send_model())
  
      client.train(epochs=cloud.EPOCHS, batch_size=cloud.BATCH_SIZE)

      val_acc_epoch_start.append(client.model.history.history['val_acc'][0])
      val_acc_epoch_end.append(client.model.history.history['val_acc'][cloud.EPOCHS-1])
  
      ''' pass trained weights to aggregator '''
      aggregator.get_weights_from_clients(client_idx=client.idx, weights=client.get_weights_from_model())
  
    ''' AFTER CLIENTS ARE DONE TRAINING IN THIS GEN '''
    ''' aggregator does his thing and sends the aggregated weights over to cloud '''
    aggregator.send_weights_to_cloud(cloud)
    print('Round {}: Generation {}: val_acc_epoch_s: {}'.format(i+1, j+1, sum(val_acc_epoch_start)/len(val_acc_epoch_start)))
    print('Round {}: Generation {}: val_acc_epoc_e: {}'.format(i+1, j+1, sum(val_acc_epoch_end)/len(val_acc_epoch_end)))
    val_acc_epoch_start = []
    val_acc_epoch_end = []

--- Round 1 ---
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

---- Generation 1. For Client 0 ----
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

---- Generation 1. For Client 1 ----
Round 1: Generation 1: val_acc_epoch_s: 0.8123999834060669
Round 1: Generation 1: val_acc_epoc_e: 0.9368000030517578

---- Generation 2. For Client 0 ----

---- Generation 2. For Client 1 ----
Round 1: Generation 2: val_acc_epoch_s: 0.7769999802112579
Round 1: Generation 2: val_acc_epoc_e: 0.9133999943733215

---- Generation 3. For Client 0 ----

---- Generation 3. For Client 1 ----
Round 1: Generation 3: val_acc_epoch_s: 0.800000011920929
Round 1: Generation 3: val_acc_epoc_e: 0.9087999761104584
--- Round 2 ---

---- Generation 1. For Client 0 ----


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-d9ea5d97a3a3>", line 24, in <module>
    client.train(epochs=cloud.EPOCHS, batch_size=cloud.BATCH_SIZE)
  File "<ipython-input-5-77fc3f7a139f>", line 23, in train
    self.model.fit(self.train_data, self.train_labels, validation_data=(self.test_data, self.test_labels), epochs=epochs, batch_size = batch_size, verbose=0)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/keras/engine/training.py", line 727, in fit
    use_multiprocessing=use_multiprocessing)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/keras/engine/training_arrays.py", line 675, in fit
    steps_name='steps_per_epoch')
  File "/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/keras/engine/training_arrays.py", line 394, in model_iteration
    batch_out

KeyboardInterrupt: ignored